<a href="https://colab.research.google.com/github/Kavish1504/Data_Generation_Using_Modelling_And_Simulation/blob/main/Data_Generation_Using_Modelling_And_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gekko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 68.5 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from gekko import GEKKO
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, root_mean_squared_error
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor

def simulate_population(r, K):
    try:
        m = GEKKO(remote = False)
        m.time = np.linspace(0, 5, 50)
        P = m.Var(value = 10, lb = 0)
        m.Equation(P.dt() == r * P * (1 - P/K))
        m.options.IMODE = 4
        m.options.NODES = 3
        m.options.SOLVER = 1
        m.solve(disp = False)
        final_population = P.value[-1]
        max_population = max(P.value)
        avg_population = np.mean(P.value)
        return final_population, max_population, avg_population
    except:
        return None

parameter_bounds = {
    "r": (0.1, 2.0),
    "K": (50, 500)
}

records = []
while len(records) < 1000:
    r = np.random.uniform(*parameter_bounds["r"])
    K = np.random.uniform(*parameter_bounds["K"])
    result = simulate_population(r, K)
    if result is not None:
        final_p, max_p, avg_p = result
        records.append([r, K, final_p, max_p, avg_p])

df = pd.DataFrame(records, columns = ["r", "K","final_population","max_population","avg_population"])
print(df.head(20))

X = df[["r", "K"]]
y = df["final_population"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

models = {
    "LinearRegression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "DecisionTree": DecisionTreeRegressor(),
    "RandomForest": RandomForestRegressor(),
    "GradientBoosting": GradientBoostingRegressor(),
    "SVR": SVR(),
    "KNN": KNeighborsRegressor()
}

results = []
for name, model in models.items():
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    r2 = r2_score(y_test, preds)
    rmse = root_mean_squared_error(y_test, preds)
    mse = mean_squared_error(y_test, preds)
    results.append([name, r2, rmse, mse])

comparison_df = pd.DataFrame(results, columns = ["Model","R2 Score","RMSE","MSE"])
comparison_df = comparison_df.sort_values(by = "R2 Score", ascending = False)

print("\nModel Comparison:\n", comparison_df)
print("\nBest Model:\n", comparison_df.iloc[0])

           r           K  final_population  max_population  avg_population
0   1.684018  403.173529        399.699310      399.699310      226.085795
1   1.757874  338.443194        336.751870      336.751870      202.391055
2   0.103317  150.097566         16.040222       16.040222       12.809643
3   0.617641  321.847833        132.900635      132.900635       52.610158
4   0.735260  434.104654        209.314830      209.314830       75.664817
5   1.395971  131.470062        129.999129      129.999129       82.892396
6   0.376134  216.767667         52.194253       52.194253       26.405171
7   1.750535  298.317153        296.959081      296.959081      182.123496
8   0.563217  136.524029         77.698461       77.698461       37.264229
9   1.213506  485.730837        437.483310      437.483310      184.014788
10  1.995663  409.037120        408.276296      408.276296      255.951436
11  1.436679  270.503946        265.251806      265.251806      146.852760
12  0.554367  251.337339 